# Analisador de Sentimentos

## Parte 1

O objetivo deste _notebook_ é definir um método capaz de identificar se o sentimento expresso em um dado texto é positivo ou negativo. Utilizaremos _reviews_ de filmes extraídos do site [_Rotten Tomatoes_](https://www.rottentomatoes.com/) e baseados no _dataset_ disponível [neste site](http://www.cs.cornell.edu/people/pabo/movie-review-data/).

Nossa __hipótese__ é que o significado de um texto está diretamente relacionado às palavras que o compõe. Para verificar esta hipótese podemos analisar a frequência com que certas palavras são utilizadas. Palavras como _good_ e _best_ devem ser mais frequentes em _reviews_ positivos, já palavras como _bad_ e _worst_ devem ser mais utilizadas em _reviews_ negativos. Além de analisar a frequência de cada palavra, podemos verificar a frequência de conjuntos de palavras, já que, embora a expressão _not good_ contenha uma palavra positiva, seu real sentimento é negativo.

Como consequência, assumiremos que, mesmo sem conhecer a estrutura sintática e/ou semântica de um texto é possível discriminá-lo entre sentimentos positivos e negativos. Mais ainda, assumiremos que nem mesmo a ordem em que as palavras aparecem importa. Esta é uma abordagem bastante utilizada e conhecida como _Bag of Words_ (_unigram_ para modelagens com palavras únicas e _n-gram_ para modelagens com conjuntos de palavras) \[[1](http://www.iosrjournals.org/iosr-jce/papers/Vol16-issue1/Version-5/F016153438.pdf?id=8590)]\.

### Pré-processamento

A primeira etapa consistirá em construir um vocabulário $V = \{w_1, w_2, ..., w_N\}$, onde $w_i$ é uma palavra existente no conjunto de treinamento e que $w_i \ne w_j$ para todo $i$ e $j$. 

### Representação

Cada _review_ será representado como um vetor de tamanho $N$, sendo que cada posição $i$ conterá o número de vezes em que a palavra $w_i \in V$ apareceu no dado _review_.

### Modelagem

Os _reviews_ podem ser classificados utilizando diversos modelos como regressão logística, árvores de decisão, random forest, SVM ou redes neurais. Neste _notebook_ decidimos utilizar redes neurais por ser uma técnica que vem apresentando bons resultados em diversas áreas, incluindo processamento de linguagem natural \[[2](https://www.jair.org/media/4992/live-4992-9623-jair.pdf), [3](https://www.aaai.org/ocs/index.php/AAAI/AAAI16/paper/view/11957/12160), [4](https://arxiv.org/pdf/1606.01781.pdf), [5](http://sigport.org/documents/sentiment-analysis-recurrent-neural-network-and-unsupervised-neural-language-model)\].

Nossa rede neural terá uma camada de entrada, uma camada escondida e uma camada de saída, sendo que:

* A camada de entrada terá tamanho $N$ já que nossos _reviews_ são representados por vetores de tamanho $N$. 
* A escolha por uma única camada escondida é arbitrária* e poderia ser melhor investigada. Mas devemos tomar cuidado com relação à esse número, pois redes neurais com número excessivo de camadas escondidas podem sofrer de _overfiting_.
* A camada intermediária terá 20 neurônios. Este número também é arbitrário* e pode ser melhor investigado. Assim como o item anterior, aumentar excessivamente o número de neurônios pode levar a nossa rede a sofrer de _overfiting_.
* A camada intermediária terá ativação _relu_, que vem sendo bastante utilizada e apresentando resultados melhores do que ativações mais clássicas como _sigmoids_. Ao utilizar _relu_ o cálculo das derivas (via backpropagation) é mais simples e mais rápido além de diminuir as chances do gradiente sumir (como acontece com _sigmoids_ já que sua derivada tem como valor máximo 0.25 - ver figura abaixo)

![Função sigmoid e sua derivada](sigmoid.png)

<center>__Figura 1__: _Em azul temos a função sigmoid e em vermelho sua derivada_</center>

* A camada de saída terá 2 neurônios com ativação _softmax_, ou seja, apresentará a 'probabilidade' do _review_ ser classificado como positivo ou negativo.

_* Para evitar escolhas arbitrária para o número de camadas escondidas e de neurônios, uma opção é utilizar alguma ténica de busca, que pode ser exaustiva ou até mesmo heurísticas de otimização como algoritmo genético._

### Avaliação

Para avaliarmos nosso modelo utilizaremos a técnica _k-fold cross-validation_, sendo $k=5$. Ou seja, dividiremos aleatoriamente o dataset em 5 partes de mesmo tamanho chamadas $d_1, d_2, d_3, d_4$ e $d_5$. Para $i = [1,k]$, treinaremos o modelo com todos os $d_j$ onde $j \ne i$ e avaliaremos o modelo utilizando $d_i$.

O método de avaliação que utilizaremos é a acurácia, ou seja, a porcentagem de _reviews_ corretamente rotulados.

## Parte 2

### Carregando o Dataset

Primeiramente vamos carregar os _reviews_:

In [1]:
f = open('dataset/rt-polarity.neg','r')
negative_reviews = list(map(lambda x:x[:-1], f.readlines()))
f.close()

In [2]:
f = open('dataset/rt-polarity.pos','r')
positive_reviews = list(map(lambda x:x[:-1], f.readlines()))
f.close()

e juntá-los em um mesmo dataset único chamado `reviews`, bem como seus rótulos em `labels`:

* 0 == NEGATIVE
* 1 == POSITIVE

In [3]:
labels = [0] * len(negative_reviews) + [1] * len(positive_reviews)
reviews = negative_reviews + positive_reviews

In [4]:
print("Dataset contêm {} reviews".format(len(reviews)))

Dataset contêm 10662 reviews


### Analisando Features

Agora vamos analisar como iremos extrair as _features_ de um texto.

In [5]:
from string import punctuation
from collections import Counter
import numpy as np

Nos meus testes o melhor conjunto de features foi a utilização de features unigram (uma palavra) e bigram (duas palavras consecutivas) em conjunto:

In [6]:
def extract_features(content):
    features = []
    prev_word = None
    for word in content.split(' '):
        if word not in punctuation:
            if prev_word is not None:
                features.append(prev_word + "|" + word)
                features.append(prev_word)
                features.append(word)
            prev_word = word
    return features

Vamos avaliar as features que extraímos contando quantas vezes cada feature aparece no dataset (classificadas como positivas ou negativas):

In [7]:
positive_counter = Counter()
negative_counter = Counter()
total_counter = Counter()
for content, label in zip(reviews, labels):
    if label == 1:
        for feature in extract_features(content):
            positive_counter[feature] += 1
            total_counter[feature] +=1
    else:
        for feature in extract_features(content):
            negative_counter[feature] += 1
            total_counter[feature] +=1

In [8]:
positive_counter.most_common()[:30]

[('the', 9487),
 ('and', 7096),
 ('a', 6923),
 ('of', 6620),
 ('to', 3925),
 ('is', 3390),
 ('in', 2623),
 ('that', 2503),
 ('it', 1988),
 ('with', 1721),
 ('as', 1687),
 ('but', 1557),
 ('film', 1554),
 ('its', 1378),
 ('an', 1336),
 ('for', 1300),
 ('this', 1168),
 ('movie', 972),
 ('you', 933),
 ("it's", 899),
 ('on', 835),
 ('be', 819),
 ('has', 744),
 ('by', 739),
 ('about', 715),
 ('at', 700),
 ('of|the', 699),
 ('not', 691),
 ('are', 691),
 ('from', 690)]

In [9]:
negative_counter.most_common()[:30]

[('the', 9338),
 ('a', 6430),
 ('of', 5498),
 ('and', 5292),
 ('to', 4502),
 ('is', 3295),
 ('in', 2522),
 ('that', 2393),
 ('it', 2145),
 ('as', 1789),
 ('but', 1707),
 ('for', 1482),
 ('movie', 1427),
 ('this', 1368),
 ('with', 1313),
 ('its', 1248),
 ('film', 1190),
 ('an', 1028),
 ('be', 1020),
 ('on', 925),
 ("it's", 879),
 ('like', 839),
 ('not', 829),
 ('by', 815),
 ('more', 809),
 ('than', 764),
 ('you', 762),
 ('have', 727),
 ('are', 717),
 ('about', 715)]

Como podemos ver, as palavras mais comuns tanto no conjunto de _reviews_ positivos quanto no de negativos são parecidas. Isso já era esperado, já que a maior parte das palavras podem ser consideradas neutras.

Para podermos observar quais as palavras que melhor caracterizam _reviews_ positivos e negativos devemos calcular a razão entre suas frequências:

In [10]:
min_count = 20
sentiment_ratio = Counter()
for feature, n in list(total_counter.most_common()):
    if(n > min_count):
        sentiment_ratio[feature] = float(positive_counter[feature] + 1) / float(negative_counter[feature] + 1)

Desta forma, podemos concluir que razões:

* iguais (ou próximas) a 1: representam palavras que aparecem em _reviews_ positivos e negativos com a mesma (ou semelhante) frequência.
* maiores do que 1: representam palavras que aparecem mais em _reviews_ positivos, e portanto caracterizam melhor esse tipo de _review_
* menores do que 1: representam palavras que aparecem mais em _reviews_ negativos, e portanto caracterizam melhor esse tipo de _review_

In [11]:
sentiment_ratio.most_common()[:30]

[('riveting', 40.0),
 ('gem', 32.0),
 ('wonderfully', 31.0),
 ('lively', 29.0),
 ('detailed', 27.0),
 ('heartwarming', 27.0),
 ('polished', 25.0),
 ('vividly', 25.0),
 ('startling', 23.0),
 ('tour', 23.0),
 ('spare', 22.0),
 ('heartbreaking', 22.0),
 ('engrossing', 20.333333333333332),
 ('mesmerizing', 15.5),
 ('inventive', 15.0),
 ('refreshingly', 13.0),
 ('what|makes', 13.0),
 ('refreshing', 12.666666666666666),
 ('wonderful', 12.6),
 ('warm', 12.4),
 ('realistic', 11.0),
 ('captures', 10.8),
 ('powerful', 10.11111111111111),
 ('provides', 10.0),
 ('wry', 9.666666666666666),
 ('touching', 9.625),
 ('tender', 9.333333333333334),
 ('unexpected', 9.2),
 ('a|compelling', 9.0),
 ('chilling', 9.0)]

In [12]:
list(reversed(sentiment_ratio.most_common()))[:30]

[('unfunny', 0.02),
 ('badly', 0.02127659574468085),
 ('poorly', 0.02702702702702703),
 ('disguise', 0.029411764705882353),
 ('pointless', 0.03225806451612903),
 ('pinocchio', 0.037037037037037035),
 ('bore', 0.041666666666666664),
 ('uninspired', 0.041666666666666664),
 ('lousy', 0.041666666666666664),
 ('the|problem', 0.041666666666666664),
 ('plodding', 0.041666666666666664),
 ('lifeless', 0.043478260869565216),
 ('product', 0.043478260869565216),
 ('incoherent', 0.045454545454545456),
 ('flat', 0.056338028169014086),
 ('mediocre', 0.061224489795918366),
 ('mindless', 0.06451612903225806),
 ('generic', 0.06666666666666667),
 ('boring', 0.06741573033707865),
 ('routine', 0.07317073170731707),
 ('90', 0.075),
 ('disaster', 0.08),
 ('dull', 0.0873015873015873),
 ('supposed|to', 0.09523809523809523),
 ('ends|up', 0.09523809523809523),
 ('stale', 0.0967741935483871),
 ('tiresome', 0.1),
 ('stupid', 0.1),
 ('the|worst', 0.10256410256410256),
 ('offensive', 0.10344827586206896)]

Como esperado, palavras como _wonderfully_ e _gem_ caracterizam _reviews_ positivos, e palavras como _unfunny_ e _badly_ caracterizam _reviews_ negativos.

### Extraindo Features

Vamos definir algumas funções para facilitar a extração de features.

Primeiramente, vamos definir a função `build_vocabulary` que retorna um vocabulário contendo todas as palavras dos `reviews` com frequência maior do que `cutoff`. Além disso, também retorna `feature2index` que será utilizado para encontrar o número do índice de uma dada feature.

In [13]:
def build_vocabulary(reviews, cutoff):
    counter = Counter()
    for content in reviews:
        for feature in extract_features(content):
            counter[feature] += 1
    vocabulary = []
    feature2index = Counter()
    for f in counter.keys():
        if counter[f] > cutoff:
            vocabulary.append(f)
            feature2index[f] = len(vocabulary) - 1
    return (vocabulary, feature2index)

Finalmente, vamos definir uma função que recebe um vocabulário e um conversor `feature2index` e transforma os _reviews_ em _Bags of Words_:

In [14]:
def build_dataset(reviews, vocabulary, feature2index):
    feature_dataset = np.zeros((len(reviews), len(vocabulary)))
    i = 0
    for content in reviews:
        for f in extract_features(content):
            feature_dataset[i][feature2index[f]] += 1
        i += 1
    return feature_dataset

### Implementando o Modelo

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


A seguir definimos uma função que constrói nossa rede neural completamente conectada:

* Uma camada de entrada de tamanho N = tamanho do vocabulário
* Uma camada oculta com 20 neurônios, com função de ativação ReLu
* Uma camada de saída com 2 neurônios com função de ativação softmax

Durante o treinamento, será otimizada a função `categorical_crossentropy`, utilizando o algoritmo de otimização `adam`. A cada iteração será apresentada a acurácia de predição do próprio conjunto de treinamento.

In [16]:
def create_model():
    model = Sequential()
    model.add(Dense(units=20, input_dim=len(vocabulary)))
    model.add(Activation('relu'))
    model.add(Dense(units=2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Utilizaremos o método _5-fold cross-validation_ para avaliar nosso modelo.

In [17]:
from sklearn.model_selection import StratifiedKFold

Os hiper-parâmetros utilizados para o treinamento da rede neural são:

* batch_size
* epochs - O número de iterações escolhido foi de 10. Acima disso a rede parece iniciar a sofrer de overffiting
* cutoff - A frequência mínima de uma palavra no conjunto de treinamento para que ela entre no vocabulario é de 20. Números menores do que esse melhoram o desempenho da rede neural, mas o tempo de execução aumenta bastante.

In [18]:
batch_size = 128
epochs = 10
cutoff = 20

X = np.array(reviews)
y = np.array(labels)

kfold = StratifiedKFold(n_splits=5, shuffle=True)
evaluations = []
k = 1
for train_index, test_index in kfold.split(X, y):
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    vocabulary, feature2index = build_vocabulary(X_train, cutoff)
    X_train = build_dataset(X_train, vocabulary, feature2index)
    y_train = keras.utils.to_categorical(y_train, 2)
    X_test = build_dataset(X_test, vocabulary, feature2index)
    y_test = keras.utils.to_categorical(y_test, 2)
    
    model = create_model()
    print("Fold {}/5".format(k))
    model.fit(X_train, y_train, batch_size=batch_size, verbose=1, epochs=epochs)
    print("Evaluating:")
    result = model.evaluate(X_test, y_test)
    evaluations.append(result[1])
    print(" - loss: {} - acc: {}".format(result[0], result[1]))
    print("******************************************************************")
    k += 1

Fold 1/5
Epoch 1/10
8528/8528 [==============================] - 0s - loss: 0.6570 - acc: 0.6136     
Epoch 2/10
8528/8528 [==============================] - 0s - loss: 0.5514 - acc: 0.7462     
Epoch 3/10
8528/8528 [==============================] - 0s - loss: 0.4781 - acc: 0.7913     
Epoch 4/10
8528/8528 [==============================] - 0s - loss: 0.4276 - acc: 0.8168     
Epoch 5/10
8528/8528 [==============================] - 0s - loss: 0.3879 - acc: 0.8340     
Epoch 6/10
8528/8528 [==============================] - 0s - loss: 0.3576 - acc: 0.8494     
Epoch 7/10
8528/8528 [==============================] - 0s - loss: 0.3286 - acc: 0.8644     
Epoch 8/10
8528/8528 [==============================] - 0s - loss: 0.3015 - acc: 0.8790     
Epoch 9/10
8528/8528 [==============================] - 0s - loss: 0.2770 - acc: 0.8891     
Epoch 10/10
8528/8528 [==============================] - 0s - loss: 0.2556 - acc: 0.8985     
Evaluating:
1600/2134 [=====================>........] - ETA

In [19]:
print("Acurácia média {} +/- {}".format(np.array(evaluations).mean(), np.array(evaluations).std()))

Acurácia média 0.7302600089932405 +/- 0.008616289211280678
